In [2]:
# 1. SETUP AND INSTALLATION
# -------------------------
# Using the exact setup from your working script.
print("Installing required packages (transformers from source)...")
!pip install -q timm --upgrade
!pip install -q accelerate
!pip install -q git+https://github.com/huggingface/transformers.git

import os
import time
import torch
import kagglehub
# FIX: Reverting to AutoModelForCausalLM as per your working example
from transformers import AutoTokenizer, AutoModelForCausalLM 
from typing import List

# These environment settings are from your working script.
# While `disable_compile=True` is the main fix used later, we will keep these.
print("🔄 Configuring PyTorch backend for compatibility...")
try:
    import torch._dynamo
    torch._dynamo.reset() 
    os.environ["TORCH_COMPILE_BACKEND"] = "eager"
    torch._dynamo.config.suppress_errors = True
except ImportError:
    print("⚠️ Could not import torch._dynamo. Skipping dynamo configuration.")

print("✅ Packages installed successfully!")


# 2. MODEL CONFIGURATION
# ----------------------
print("Downloading model files with kagglehub...")
MODEL_PATH = kagglehub.model_download("google/gemma-3n/transformers/gemma-3n-e2b-it")
print(f"✅ Model path set to: {MODEL_PATH}")


# 3. GENERAL-PURPOSE TRANSLATOR CLASS
# -----------------------------------
# This class is based directly on your working EnglishToItalianTranslator.
# It has been generalized to handle multiple languages.
class GemmaTranslator: # CHANGED: Renamed class
    def __init__(self, model_path):
        self.model_path = model_path
        self.tokenizer = None
        self.model = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self._initialize_model()

    def _initialize_model(self):
        """Loads the Gemma model using AutoModelForCausalLM."""
        if not os.path.exists(self.model_path):
            print("❌ Cannot initialize model: Path does not exist.")
            return

        try:
            print("🔄 Loading tokenizer...")
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
            
            print("🔄 Loading model... (This may take a moment)")
            # FIX: Using AutoModelForCausalLM as it correctly handles the 'gemma3n' model type.
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_path,
                torch_dtype=torch.float16,
            ).to(self.device)

            print("✅ Translator initialized successfully!")
            if self.model:
                  print(f"📊 Model configured for device: {self.device}")

        except Exception as e:
            print(f"❌ Error initializing model: {e}")
            self.model = None
            self.tokenizer = None
            
    # CHANGED: This method now accepts a `target_language` argument.
    def translate(self, english_text: str, target_language: str) -> str:
        """
        Translates English text into the specified target language.
        """
        if not self.model or not self.tokenizer:
            return "❌ Model not initialized. Please check for errors above."

        # The prompt is now dynamic based on the target_language.
        prompt = f"""Translate the following English text to {target_language}. Provide only the {target_language} translation.

English: "{english_text}"
{target_language}:"""

        try:
            start_time = time.time()
            
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
            
            # This is the key fix for GPU compatibility.
            outputs = self.model.generate(**inputs, max_new_tokens=100, disable_compile=True)
            
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            end_time = time.time()
            
            # The response parsing is now also dynamic.
            if f"{target_language}:" in response:
                translation = response.split(f"{target_language}:")[-1].strip()
            else:
                translation = response[len(prompt) - len(f'\n{target_language}:'):].strip()
            
            translation = translation.replace('"', '').strip()
            
            print(f"⏱️  '{english_text[:30]}...' -> {target_language} in {end_time - start_time:.2f}s")
            return translation

        except Exception as e:
            return f"❌ Translation error: {e}"


# 4. INITIALIZE AND TEST THE MULTILINGUAL TRANSLATOR
# ----------------------------------------------------
print("\n--- Initializing Translator ---")
translator = GemmaTranslator(MODEL_PATH)
print("-----------------------------\n")

if translator.model:
    # A dictionary of sentences to test the new multilingual capability.
    test_cases = {
        "Where is the nearest library?": "Spanish",
        "How much does this cost?": "German",
        "I would like to order a coffee.": "French",
        "This is a very powerful language model.": "Italian",
    }

    print("--- Running Multilingual Translation Examples ---")
    for sentence, language in test_cases.items():
        translation = translator.translate(sentence, language)
        print(f"🇺🇸 English: {sentence}")
        print(f"🌍 {language}: {translation}")
        print("-" * 40)
        
    print("\n🎉 All tasks complete!")
else:
    print("Could not run translation examples because the model failed to initialize.")

Installing required packages (transformers from source)...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
🔄 Configuring PyTorch backend for compatibility...
✅ Packages installed successfully!
✅ Model path set to: /kaggle/input/gemma-3n/transformers/gemma-3n-e2b-it/1

--- Initializing Translator ---
🔄 Loading tokenizer...
🔄 Loading model... (This may take a moment)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Translator initialized successfully!
📊 Model configured for device: cuda
-----------------------------

--- Running Multilingual Translation Examples ---
⏱️  'Where is the nearest library?...' -> Spanish in 2.59s
🇺🇸 English: Where is the nearest library?
🌍 Spanish: ¿Dónde está la biblioteca más cercana?
----------------------------------------
⏱️  'How much does this cost?...' -> German in 13.54s
🇺🇸 English: How much does this cost?
🌍 German: Ent
----------------------------------------
⏱️  'I would like to order a coffee...' -> French in 1.23s
🇺🇸 English: I would like to order a coffee.
🌍 French: Je voudrais commander un café.
----------------------------------------
⏱️  'This is a very powerful langua...' -> Italian in 1.47s
🇺🇸 English: This is a very powerful language model.
🌍 Italian: Questo è un modello linguistico molto potente.
----------------------------------------

🎉 All tasks complete!
